In [1]:
import pandas as pd
from scipy import stats
import scikit_posthocs as sp

**Functions**

In [35]:
def prep_rel_abund_tables(biom_table,
                          metadata_table,
                          day_filter):
    day_filt_meta = metadata_table.loc[metadata_table["day_post_inf"] == day_filter]
    wanted_df = day_filt_meta[["sampleid"]].merge(biom_table, how="left")
    wanted_df = pd.melt(wanted_df, id_vars="sampleid")
    wanted_df = wanted_df.rename(columns={"variable":"asv", "value":"abund"})
    wanted_df = metadata_table[["sampleid", "mouse_id", "diet", "vendor"]].merge(wanted_df, how="right", on=["sampleid"])
    wanted_df["sum_abund"] = wanted_df.groupby("sampleid")["abund"].transform('sum')
    wanted_df["rel_abund"] = wanted_df["abund"] / wanted_df["sum_abund"]
    return(wanted_df)

def merge_tables_tax(input_table1,
                     input_table2,
                     rename_merged_cols,
                     tax_table,
                     tax_col_list,
                     removed_cols):
    ## merged dataframe
    merged_table = input_table1.merge(input_table2, how="left", on=["asv", "mouse_id"])
    merged_table = merged_table.rename(columns=rename_merged_cols)
    merged_table = merged_table.drop(["diet_y", "vendor_y"], axis=1)
    ## taxonomic results
    tax_table = tax_table.rename(columns={"Feature ID":"asv", "Taxon":"taxon", "Confidence":"conf"})
    ## putting them together
    mergedTax_table = merged_table.merge(tax_table, how="left", on=["asv"])
    ## creating separate columns for different taxonomic levels
    mergedTax_table[tax_col_list] = mergedTax_table["taxon"].str.split(";",expand=True)
    ## removing extra columns from the merged taxonomic table that I don't want
    mergedTax_table = mergedTax_table[mergedTax_table.columns.difference(removed_cols)] 
    return(mergedTax_table)

def find_tax_differences(mergedTax_table,
                         wanted_variable_col,
                         wanted_tax_col,
                         bloom_greater_than,
                         baseline_less_than):
    output_list = []
    ## pulling only the columns I want from the merged taxonomic table
    tax_diff_table = mergedTax_table.loc[:, ("rel_abund_baseline", "rel_abund_bloomDay", 
                                             wanted_variable_col, wanted_tax_col)]
    ## calculating deltas
    tax_diff_table['rel_abund_diff_d3-d15'] = tax_diff_table['rel_abund_bloomDay'] - tax_diff_table['rel_abund_baseline']
    ## summarizing my deltas by adding them up per taxonomic classification 
    sumDeltas_tax_diff = tax_diff_table.groupby(by=[wanted_tax_col, wanted_variable_col], as_index=False).agg({'rel_abund_diff_d3-d15':'sum'})
    ## finding microbes that most likely were blooming at day 3
    bloom_tax_diff = sumDeltas_tax_diff.loc[sumDeltas_tax_diff["rel_abund_diff_d3-d15"] > bloom_greater_than]
    ## finding microbes that were much higher at baseline than they were at day 3
    baseline_tax_diff = sumDeltas_tax_diff.loc[sumDeltas_tax_diff["rel_abund_diff_d3-d15"] < baseline_less_than]

    ## adding dataframe outputs to a list
    output_list.append(sumDeltas_tax_diff)
    output_list.append(bloom_tax_diff)
    output_list.append(baseline_tax_diff)
    return(output_list)


**File Paths**

In [3]:
tax_list = ["tax_domain", "tax_phylum", "tax_class", "tax_order", "tax_family", "tax_genus", "tax_species"]
remove_from_mergedTax = ['sum_abund_baseline', 'sum_abund_bloomDay', 'taxon', 'conf']
new_merged_df_names = {"sampleid_x":"sampleid_baseline", "abund_x":"abund_baseline", "sum_abund_x":"sum_abund_baseline",     
                       "rel_abund_x":"rel_abund_baseline", "sampleid_y":"sampleid_bloomDay", "abund_y":"abund_bloomDay",
                       "sum_abund_y":"sum_abund_bloomDay", "rel_abund_y":"rel_abund_bloomDay", "diet_x":"diet", "vendor_x":"vendor"}


otu_fp = "../../data/qiime/otu_table.tsv"
meta_fp = "../../data/misc/proc_newExp_d15-d3_metadata.tsv"
tax_fp = "../../data/qiime/taxonomy.tsv"
## parquets are for big files that you want to be read in much more quickly (it's compressed)
ml_out_parquet = "../data/newExp_ml_out.parquet"
ml_out_tsv = "../data/newExp_ml_out.tsv"

**Data Wrangling** \
For merged taxonomic relative abundance biom table!

In [4]:
biom_df = pd.read_csv(otu_fp, sep="\t")
meta_df = pd.read_csv(meta_fp, sep="\t")
tax_df = pd.read_csv(tax_fp, sep="\t")

biom_df = biom_df.rename(columns={"Unnamed: 0":"sampleid"})

In [5]:
## baseline microbiome relative abundance data wrangling
baseline_df = prep_rel_abund_tables(biom_table=biom_df,
                                    metadata_table=meta_df,
                                    day_filter=-15)

## bloom day relative abundance data wrangling
bloomDay_df = prep_rel_abund_tables(biom_table=biom_df,
                                    metadata_table=meta_df,
                                    day_filter=3)

In [6]:
## joining the two above tables together with taxonomic information 
mergedTax_df = merge_tables_tax(input_table1=baseline_df,
                                input_table2=bloomDay_df,
                                rename_merged_cols=new_merged_df_names,
                                tax_table=tax_df,
                                tax_col_list=tax_list,
                                removed_cols=remove_from_mergedTax)
mergedTax_df

,abund_baseline,abund_bloomDay,asv,diet,mouse_id,rel_abund_baseline,rel_abund_bloomDay,sampleid_baseline,sampleid_bloomDay,tax_class,tax_domain,tax_family,tax_genus,tax_order,tax_phylum,tax_species,vendor
0,690.0,0.0,2b7b5b3f7fc005ae8c623d6d61947eca,HF/LF,CDD02.Tc.HFLF.3,0.006900,0.000000,CDD02.Tc.HFLF.3.00,CDD02.Tc.HFLF.3.18,c__Bacteroidia,d__Bacteria,f__Muribaculaceae,g__Muribaculaceae,o__Bacteroidales,p__Bacteroidota,s__uncultured_Bacteroidales,taconic
1,1504.0,0.0,2b7b5b3f7fc005ae8c623d6d61947eca,LF/LF,CDD02.Tc.LFLF.4,0.015040,0.000000,CDD02.Tc.LFLF.4.00,CDD02.Tc.LFLF.4.18,c__Bacteroidia,d__Bacteria,f__Muribaculaceae,g__Muribaculaceae,o__Bacteroidales,p__Bacteroidota,s__uncultured_Bacteroidales,taconic
2,1135.0,0.0,2b7b5b3f7fc005ae8c623d6d61947eca,Chow,CDD02.Tc.Chow.4,0.011351,0.000000,CDD02.Tc.Chow.4.00,CDD02.Tc.Chow.4.18,c__Bacteroidia,d__Bacteria,f__Muribaculaceae,g__Muribaculaceae,o__Bacteroidales,p__Bacteroidota,s__uncultured_Bacteroidales,taconic
3,1092.0,178.0,2b7b5b3f7fc005ae8c623d6d61947eca,Chow,CDD02.Tc.Chow.2,0.010920,0.001788,CDD02.Tc.Chow.2.00,CDD02.Tc.Chow.2.18,c__Bacteroidia,d__Bacteria,f__Muribaculaceae,g__Muribaculaceae,o__Bacteroidales,p__Bacteroidota,s__uncultured_Bacteroidales,taconic
4,1541.0,0.0,2b7b5b3f7fc005ae8c623d6d61947eca,Chow,CDD02.Tc.Chow.1,0.015410,0.000000,CDD02.Tc.Chow.1.00,CDD02.Tc.Chow.1.18,c__Bacteroidia,d__Bacteria,f__Muribaculaceae,g__Muribaculaceae,o__Bacteroidales,p__Bacteroidota,s__uncultured_Bacteroidales,taconic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56245,0.0,0.0,8f1a7336ff8430574c3cbd890815fad5,LF/HF,CDD02.CR.LFHF.1,0.000000,0.000000,CDD02.CR.LFHF.1.00,CDD02.CR.LFHF.1.18,c__Bacilli,d__Bacteria,f__Erysipelotrichaceae,g__[Clostridium]_innocuum_group,o__Erysipelotrichales,p__Firmicutes,None,charles_river
56246,0.0,0.0,8f1a7336ff8430574c3cbd890815fad5,LF/HF,CDD02.CR.LFHF.3,0.000000,0.000000,CDD02.CR.LFHF.3.00,CDD02.CR.LFHF.3.18,c__Bacilli,d__Bacteria,f__Erysipelotrichaceae,g__[Clostridium]_innocuum_group,o__Erysipelotrichales,p__Firmicutes,None,charles_river
56247,0.0,0.0,8f1a7336ff8430574c3cbd890815fad5,HF/LF,CDD02.CR.HFLF.4,0.000000,0.000000,CDD02.CR.HFLF.4.00,CDD02.CR.HFLF.4.18,c__Bacilli,d__Bacteria,f__Erysipelotrichaceae,g__[Clostridium]_innocuum_group,o__Erysipelotrichales,p__Firmicutes,None,charles_river
56248,0.0,0.0,8f1a7336ff8430574c3cbd890815fad5,LF/HF,CDD02.CR.LFHF.4,0.000000,0.000000,CDD02.CR.LFHF.4.00,CDD02.CR.LFHF.4.18,c__Bacilli,d__Bacteria,f__Erysipelotrichaceae,g__[Clostridium]_innocuum_group,o__Erysipelotrichales,p__Firmicutes,None,charles_river


In [12]:
##mergedTax_df.to_parquet(ml_out_parquet)
##mergedTax_df.to_csv(ml_out_tsv, sep="\t")

**Data Wrangling** \
To identify taxonomic groups that are blooming at day 3

to identify bloomers:
- run non-parametric ANOVA (kruskal-wallis and dunn's post hoc test) on relative abundance at day -15 and day 3 grouped by mouse id and taxonomic family
- significant p-values (i.e. significant differences between the relative abundance at day -15 and day 3) can help to indicate bloomers for specific mouse ids

what I actually ended up doing:
- created a column of deltas by subtracting the relative abundance and day 3 by the relative abundance at day -15 
- summarized the deltas column by taxonomic family (added up all deltas for the same taxonomic family)
- the largest positive values are the taxonomic families that were the highest at day 3 (bloom day)
- the largest negative values are the taxonomic families that were the highest at day -15 (baseline)

what you want to do:
- be able to identify a threshold for blooming bacteria that I've identified (i.e. how do you characterize whether something is blooming or not via deltas?)
- basically what you've been doing but per each mouse instead of just vendor or diet

**By Vendor:**

In [24]:
## looking at deltas by taxonomic family
fam_outputs = find_tax_differences(mergedTax_table=mergedTax_df,
                                   wanted_variable_col="vendor",
                                   wanted_tax_col="tax_family",
                                   bloom_greater_than=1,
                                   baseline_less_than=-1)

## pop(0) pulls out the dataframe indexed at 0 (so the first one), as you pull multiple dfs out, the ones left move down in index value
## which is why you're pulling 0 each time 
fam_sumBloom_df = fam_outputs.pop(0)
fam_sumBloom_df

,tax_family,vendor,rel_abund_diff_d3-d15
0,f__AKAU3644,charles_river,0.000000
1,f__AKAU3644,taconic,0.000102
2,f__Acholeplasmataceae,charles_river,-0.008544
3,f__Acholeplasmataceae,taconic,-0.006201
4,f__Akkermansiaceae,charles_river,1.044201
...,...,...,...
99,f__Xanthomonadaceae,taconic,0.000151
100,f__[Eubacterium]_coprostanoligenes_group,charles_river,-0.327382
101,f__[Eubacterium]_coprostanoligenes_group,taconic,-0.076865
102,f__uncultured,charles_river,-0.000130


in the code chunk below I am identifying the taxonomic families that have the highest cumulative deltas, i.e. this means that those taxonomic families are most likely the ones that are blooming at day 3. I only chose cumulative values greater than 1 because there was a large separation between the families that had a cumulative value less than 1 and those that had a cumulative value greater than 1.
- tannerellaceae (9.887942) and enterobacteriaceae (8.285715) have the highest overall values (this is expected based on the taxonomic biplot I created)
- bacteroidaceae (5.408029) and enterococcaceae (5.124188) have the next highest overall values
- erysipelotrichaceae (4.246136) and morganellaceae (4.155118) come next 
- akkermansiaceae (1.825368) and peptostreptococcaceae (1.764088) are last (by a lot)

In [25]:
bloom_families = fam_outputs.pop(0)
bloom_families

,tax_family,vendor,rel_abund_diff_d3-d15
4,f__Akkermansiaceae,charles_river,1.044201
17,f__Bacteroidaceae,taconic,4.636890
39,f__Enterobacteriaceae,taconic,7.985323
40,f__Enterococcaceae,charles_river,3.674331
41,f__Enterococcaceae,taconic,1.449857
44,f__Erysipelotrichaceae,charles_river,1.214217
45,f__Erysipelotrichaceae,taconic,3.031919
62,f__Morganellaceae,charles_river,4.155158
73,f__Peptostreptococcaceae,taconic,1.118038
92,f__Tannerellaceae,charles_river,9.330909


in the code chunk below I am identifying the taxonomic families that have the lowest cumulative deltas, meaning these families are the ones that were much more present at baseline (day -15) than they were at day 3. I only chose cumulative values lower than -1 because there was a large separation between the families that had a cumulative value greater than -1 and those that had a cumulative value less than -1.
- lactobacillaceae (-14.996638) and lachnospiraceae (-12.994227) had the lowest overall values meaning that they were lost the most during the experimental timeline
- muribaculaceae (-7.989504) was the next lowest
- clostridia_UCG-014 (-1.449523) is the next lowest (by a lot)

In [26]:
baseline_families = fam_outputs.pop(0)
baseline_families

,tax_family,vendor,rel_abund_diff_d3-d15
50,f__Lachnospiraceae,charles_river,-5.610338
51,f__Lachnospiraceae,taconic,-7.383889
52,f__Lactobacillaceae,charles_river,-9.200764
53,f__Lactobacillaceae,taconic,-5.795873
64,f__Muribaculaceae,charles_river,-4.131114
65,f__Muribaculaceae,taconic,-3.858390


running the same analysis as above based on genus instead of family

In [27]:
## looking at deltas by taxonomic genus
genus_outputs = find_tax_differences(mergedTax_table=mergedTax_df,
                                     wanted_variable_col="vendor",
                                     wanted_tax_col="tax_genus",
                                     bloom_greater_than=1,
                                     baseline_less_than=-1)

## pop(0) pulls out the dataframe indexed at 0 (so the first one), as you pull multiple dfs out, the ones left move down in index value
## which is why you're pulling 0 each time 
genus_sumBloom_df = genus_outputs.pop(0)
genus_sumBloom_df

,tax_genus,vendor,rel_abund_diff_d3-d15
0,g__A2,charles_river,-0.098247
1,g__A2,taconic,-0.158681
2,g__AKAU3644,charles_river,0.000000
3,g__AKAU3644,taconic,0.000102
4,g__ASF356,charles_river,-0.031193
...,...,...,...
205,g__[Eubacterium]_ventriosum_group,taconic,-0.120798
206,g__[Eubacterium]_xylanophilum_group,charles_river,-0.335201
207,g__[Eubacterium]_xylanophilum_group,taconic,-0.295103
208,g__uncultured,charles_river,-0.557191


In [28]:
bloom_genera = genus_outputs.pop(0)
bloom_genera

,tax_genus,vendor,rel_abund_diff_d3-d15
12,g__Akkermansia,charles_river,1.044201
33,g__Bacteroides,taconic,4.636890
55,g__Clostridioides,taconic,1.118038
70,g__Enterococcus,charles_river,3.674331
71,g__Enterococcus,taconic,1.449857
81,g__Escherichia-Shigella,taconic,7.985323
152,g__Proteus,charles_river,4.155158
192,g__[Clostridium]_innocuum_group,charles_river,1.229529
193,g__[Clostridium]_innocuum_group,taconic,3.112655


In [29]:
baseline_genera = genus_outputs.pop(0)
baseline_genera

,tax_genus,vendor,rel_abund_diff_d3-d15
113,g__Lachnospiraceae_NK4A136_group,taconic,-2.180395
126,g__Lactobacillus,charles_river,-9.200764
127,g__Lactobacillus,taconic,-5.795873
136,g__Muribaculaceae,charles_river,-4.131114
137,g__Muribaculaceae,taconic,-3.858390


**By Mouse ID (as it should be for the ml models):**

not sure if this is right to do but currently I'm taking the top quartile of the summed deltas per family and mouse id to identify mice than have blooms in certain microbes.
- in the case of taxonomic families, the bloomers top quartile starts at 0.25 and goes until around 0.75 (the top of the data)
- the baseline "bottom" (top quartile since its negative numbers) starts at -0.25 and goes until around -0.65

**note:**
- **the top quartile includes almost all of the mice in this set of experiments so I'm wondering if I need to tighten my definition of how large of a delta indicates a bloom - I'm thinking about taking the mouse ids that have a difference of 0.5 and above instead of 0.25 and above**

In [39]:
## looking at deltas by taxonomic family
mouseFam_outputs = find_tax_differences(mergedTax_table=mergedTax_df,
                                        wanted_variable_col="mouse_id",
                                        wanted_tax_col="tax_family",
                                        bloom_greater_than=0.50,
                                        baseline_less_than=-0.50)

## pop(0) pulls out the dataframe indexed at 0 (so the first one), as you pull multiple dfs out, the ones left move down in index value
## which is why you're pulling 0 each time 
mouseFam_sumBloom_df = mouseFam_outputs.pop(0)
mouseFam_sumBloom_df
mouseFam_sumBloom_df

,tax_family,mouse_id,rel_abund_diff_d3-d15
0,f__AKAU3644,CDD02.CR.Chow.1,0.0
1,f__AKAU3644,CDD02.CR.Chow.2,0.0
2,f__AKAU3644,CDD02.CR.Chow.3,0.0
3,f__AKAU3644,CDD02.CR.Chow.4,0.0
4,f__AKAU3644,CDD02.CR.Chow.5,0.0
...,...,...,...
2595,f__uncultured,CDD02.Tc.LFLF.1,0.0
2596,f__uncultured,CDD02.Tc.LFLF.2,0.0
2597,f__uncultured,CDD02.Tc.LFLF.3,0.0
2598,f__uncultured,CDD02.Tc.LFLF.4,0.0


In [40]:
mouse_bloom_families = mouseFam_outputs.pop(0)
mouse_bloom_families

,tax_family,mouse_id,rel_abund_diff_d3-d15
403,f__Bacteroidaceae,CDD02.CR.Chow.4,0.553230
425,f__Bacteroidaceae,CDD02.Tc.Chow.1,0.626488
428,f__Bacteroidaceae,CDD02.Tc.Chow.4,0.534085
445,f__Bacteroidaceae,CDD02.Tc.LFLF.1,0.538659
447,f__Bacteroidaceae,CDD02.Tc.LFLF.3,0.550971
448,f__Bacteroidaceae,CDD02.Tc.LFLF.4,0.671519
449,f__Bacteroidaceae,CDD02.Tc.LFLF.5,0.550178
981,f__Enterobacteriaceae,CDD02.Tc.HFHF.2,0.657387
982,f__Enterobacteriaceae,CDD02.Tc.HFHF.3,0.525765
985,f__Enterobacteriaceae,CDD02.Tc.HFLF.1,0.595960


In [41]:
mouse_baseline_families = mouseFam_outputs.pop(0)
mouse_baseline_families

,tax_family,mouse_id,rel_abund_diff_d3-d15
1264,f__Lachnospiraceae,CDD02.CR.HFLF.5,-0.516000
1276,f__Lachnospiraceae,CDD02.Tc.Chow.2,-0.567780
1278,f__Lachnospiraceae,CDD02.Tc.Chow.4,-0.646835
1284,f__Lachnospiraceae,CDD02.Tc.HFHF.5,-0.533413
1287,f__Lachnospiraceae,CDD02.Tc.HFLF.3,-0.649724
1298,f__Lachnospiraceae,CDD02.Tc.LFLF.4,-0.579352
1317,f__Lactobacillaceae,CDD02.CR.LFHF.3,-0.513254
1322,f__Lactobacillaceae,CDD02.CR.LFLF.3,-0.568799


**Saving my Outputs:**

In [42]:
mouseFam_sumBloom_df.to_csv("../data/family_relabundDeltas.tsv", sep="\t")
mouse_bloom_families.to_csv("../data/mouseBlooms_families.tsv", sep="\t")
mouse_baseline_families.to_csv("../data/mouseBaseline_families.tsv", sep="\t")

next steps:
- figure out which taxonomic groups at baseline are important to blooms at day 3 or not - use boruta shap!!
    - done via "features" aka asvs or otus
    - throws in junk data with my data
    - if there's a diff between my data and the junk data, that means that the feature did well (if not, then the feature didn't do well, there's no info in the microbiome that can tell us what's going to happen ahead of time)
- have two classes: bloom or no bloom, can it predict which mice have blooms based on baseline microbiome?
- since you only have two classes, this could skew the results if you're not careful 
    - make rare samples more important - punish model more when it gets rare samples incorrect
    - maximize accuracy or minimize error 
    - can use measures other than accuracy to score how well your model is doing 
    - this is happening during the model.fit() command 
- input wide otu table as one side (mouse id rows, tax info in columns, fill is **relative abundance deltas**) and then whether there was a bloom or not on the other side
- can also use day 3 relative abundances to predict mortality, culture results, etc